In [1]:
!rm -rf InkubaLM-Challenge
!git clone https://github.com/melissafasol/InkubaLM-Challenge.git
%cd InkubaLM-Challenge


Cloning into 'InkubaLM-Challenge'...
remote: Enumerating objects: 468, done.
remote: Counting objects: 100% (59/59), done.
remote: Compressing objects: 100% (44/44), done.
remote: Total 468 (delta 38), reused 29 (delta 15), pack-reused 409 (from 1)
Receiving objects: 100% (468/468), 1.48 MiB | 3.87 MiB/s, done.
Resolving deltas: 100% (312/312), done.
/content/InkubaLM-Challenge


In [45]:
%cd InkubaLM-Challenge
!git pull


[Errno 2] No such file or directory: 'InkubaLM-Challenge'
/content/InkubaLM-Challenge
remote: Enumerating objects: 7, done.
remote: Counting objects: 100% (7/7), done.
remote: Compressing objects: 100% (1/1), done.
remote: Total 4 (delta 3), reused 4 (delta 3), pack-reused 0 (from 0)
Unpacking objects: 100% (4/4), 706 bytes | 353.00 KiB/s, done.
From https://github.com/melissafasol/InkubaLM-Challenge
   69465e6..03c7b22  main       -> origin/main
Updating 69465e6..03c7b22
Fast-forward
 utils/supplement.py | 62 +++++++++++++++++++++++++++++++++++---------------------------
 1 file changed, 35 insertions(+), 27 deletions(-)


In [4]:
%load_ext autoreload
%autoreload 2

In [5]:
!pip install -U trl bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 336.4/336.4 kB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.0/76.0 MB 10.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 29.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 94.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 76.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 45.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 11.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [6]:
!pip install datasets

In [46]:
import sys
sys.path.append("..")  # Add parent directory to the path

import os
from typing import List
from pathlib import Path
import numpy as np

# DO NOT EDIT
# create submission file
import pandas as pd
from huggingface_hub import login
from transformers import (
    AutoTokenizer,
)
from utils import (
    eval,
    model_function,
    supplement,
    multitask
    )

import torch
import matplotlib.pyplot as plt
from tqdm.auto import tqdm
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from peft import LoraConfig, get_peft_model, PeftModel
from datasets import load_dataset, concatenate_datasets, Dataset, Value
from trl import SFTConfig, SFTTrainer, DataCollatorForCompletionOnlyLM

In [7]:
from google.colab import drive
drive.mount('/content/drive')
output_path = "/content/drive/MyDrive/InkubaLM-Challenge/Output"
os.makedirs(output_path, exist_ok=True)

Mounted at /content/drive


In [41]:
#os.environ["TOKENIZERS_PARALLELISM"] = "false"

from huggingface_hub import login

try:
    from google.colab import userdata

    # Note: `userdata.get` is a Colab API. If you're not using Colab, set the env
    # vars as appropriate for your system.
    # userdata.get("HF_TOKEN") indicates that the name of the token in the Colab env is HF_TOKEN
    os.environ["hf_token_2"] = userdata.get("hf_token_2")
except:
    os.environ["hf_token_2"] = "----"

login(token=os.environ["hf_token_2"])

token = os.environ["hf_token_2"]
if token == "----":
    print("⚠️ Warning: No Hugging Face token found. Some models may not load.")
else:
    login(token=token)

Note: Environment variable`HF_TOKEN` is set and is the current active token independently from the token you've just configured.
Note: Environment variable`HF_TOKEN` is set and is the current active token independently from the token you've just configured.


In [11]:
hf_token_2 = '...' # paste your token here
os.environ["HF_TOKEN"] = hf_token_2

## **Load and Explore Datasets**

In [12]:
print("# Loading datasets")
se_dataset, mt_dataset,xn_dataset = multitask.load_and_combine_datasets("Train")
test_dataset = multitask.load_and_combine_datasets("Test")


# Loading datasets


README.md:   0%|          | 0.00/485 [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/39.0k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/400 [00:00<?, ? examples/s]

README.md:   0%|          | 0.00/485 [00:00<?, ?B/s]

...

Generating train split:   0%|          | 0/300 [00:00<?, ? examples/s]

All Columns: ['instruction', 'data_source', 'langs', 'ID', 'inputs', 'targets', 'premise', 'task']


Casting the dataset:   0%|          | 0/300 [00:00<?, ? examples/s]

Casting the dataset:   0%|          | 0/300 [00:00<?, ? examples/s]

Casting the dataset:   0%|          | 0/300 [00:00<?, ? examples/s]

In [35]:
se_df = pd.DataFrame(se_dataset)
mt_df = pd.DataFrame(mt_dataset)
xn_df = pd.DataFrame(xn_dataset)

hau_se_df = se_df.loc[se_df['langs'] == 'hausa']
swa_se_df = se_df.loc[se_df['langs'] == 'swahili']

hau_xnli_df = xn_df.loc[xn_df['langs'] == 'hau']
swa_xnli_df = xn_df.loc[xn_df['langs'] == 'swa']

In [41]:
print(hau_se_df['targets'].value_counts())
print(swa_se_df['targets'].value_counts())

targets
Korau          72
Tsaka-tsaki    67
Kyakkyawa      61
Name: count, dtype: int64
targets
Wastani    124
Chanya      48
Hasi        28
Name: count, dtype: int64


### **Problem:** sentiment analysis classes are heavily skewed towards neutral class for Swahili.

### **Solution**: supplement dataset with

In [14]:
xn_df['task'] = 'afrixnli'
mt_df['task'] = 'mt'

In [47]:
df = xn_df
df["is_augmented"] = False


augmented_rows = []
for _, row in df.iterrows():
    aug = supplement.apply_augmentation_xnli(row, supplement.synonyms_xnli)
    if aug is not None:
        augmented_rows.append(aug)

augmented_df = pd.DataFrame(augmented_rows)

xnli_aug_df = pd.concat([xn_df, augmented_df], ignore_index=True)

AttributeError: module 'utils.supplement' has no attribute 'apply_augmentation_xnli'

In [111]:
#sent_aug_df = sent_aug_df.drop(columns=["is_augmented"], errors="ignore")
#xnli_aug_df = xnli_aug_df.drop(columns=["is_augmented"], errors="ignore")
train_df_concat = pd.concat([xnli_aug_df, sent_aug_df, mt_df], axis=0)

In [44]:
train_df_concat = train_df_concat.drop(columns=["is_augmented"], errors="ignore")

NameError: name 'train_df_concat' is not defined

In [113]:
train_df_concat

,ID,langs,premise,inputs,instruction,targets,data_source,task
0,ID_648d37ff_dev_afrixnli_hau,hau,"Kuma kawai sai naji Hakane, wannan din ne.","Bayan nace e, ya ƙare.","Is the following question True, False or Neither?",0,,afrixnli
1,ID_f96a39cb_dev_afrixnli_hau,hau,"Eh, Eh, na sani, ba zan ma damu sosai ba idan ...",Ba zan damu ba idan ɗaukan nauyin kuɗin da ake...,"Is the following question True, False or Neither?",0,,afrixnli
2,ID_4c5d953b_dev_afrixnli_swa,swa,"Alafu, hakuelewa kabisa.","Alas, hakuweza kuelewa wazi kutokana na kizuiz...","Is the following question True, False or Neither?",1,,afrixnli
3,ID_99a61e3d_dev_afrixnli_hau,hau,"Kuma kawai sai naji Hakane, wannan din ne.",Nace a'a amma yaci gaba da ja-in-ja.,"Is the following question True, False or Neither?",2,,afrixnli
4,ID_a38aa9d8_dev_afrixnli_swa,swa,Lakini siamini chombo chochote cha algorithmic...,Kanuni haziwezi kuamua jinsi ya kutengeneza mk...,"Is the following question True, False or Neither?",1,,afrixnli
...,...,...,...,...,...,...,...,...
595,ID_b8ca5d86_dev_mt_eng-swa,eng-swa,,they must have known what was coming after int...,please convert this english content into swahili.,hawa walijua kitakachoendelea baada ya sheria ...,wmt22,mt
596,ID_7c9b5dbf_dev_mt_eng-swa,eng-swa,,a truly large amount of games and activities g...,please convert this english content into swahili.,"kwa kweli, mengi ya michezo na shughuli inakar...",wmt22,mt
597,ID_8aa26dab_dev_mt_eng-swa,eng-swa,,there are small businesses which can be succes...,could you convert this english text to swahili?,kwa hivyo vyama vidogo huenda vikafanikiwa kat...,wmt22,mt
598,ID_46056ad4_dev_mt_eng-hau,eng-hau,,identical twins are an example of natural clones.,could you convert this english text to hausa?,dabbobi da suka haifar da labaran su ne misala...,wmt22,mt


Analyse target sequence lengths (without balancing)

In [114]:
print("\n# Applying target length balancing fix")
balanced_df = multitask.balance_target_lengths(train_df_concat)

print("\n# Analyzing target sequence lengths after balancing")
task_stats_after = multitask.analyze_task_lengths(balanced_df)



# Applying target length balancing fix

# Analyzing target sequence lengths after balancing
Target sequence length analysis by task:
Task: afrixnli
count    627.0
mean      12.0
std        0.0
min       12.0
25%       12.0
50%       12.0
75%       12.0
max       12.0
Name: targets, dtype: float64
--------------------------------------------------
Task: sentiment
count    649.0
mean      12.0
std        0.0
min       12.0
25%       12.0
50%       12.0
75%       12.0
max       12.0
Name: targets, dtype: float64
--------------------------------------------------
Task: mt
count    600.000000
mean      11.801667
std        8.347163
min        2.000000
25%        6.000000
50%       10.000000
75%       15.000000
max       62.000000
Name: targets, dtype: float64
--------------------------------------------------


In [115]:
print("\n# Applying target length balancing fix")
#balanced_df = balance_target_lengths(train_df)
balanced_df = multitask.balance_target_lengths(train_df_concat)


# Applying target length balancing fix


In [116]:
balanced_dataset = Dataset.from_pandas(balanced_df.reset_index(drop=True))

In [45]:
model_name = "lelapa/InkubaLM-0.4B"
# turn off qlora: use_4bit=False
model, tokenizer, bnb_config = multitask.setup_model_and_tokenizer(model_name, token=token,use_4bit=False)

config.json:   0%|          | 0.00/763 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/2.66G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/960 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/991k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/3.95M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

In [117]:
balanced_dataset

Dataset({
    features: ['ID', 'langs', 'premise', 'inputs', 'instruction', 'targets', 'data_source', 'task'],
    num_rows: 1876
})

In [118]:
balanced_trainer = multitask.setup_trainer(
    model=model,
    dataset=balanced_dataset,
    tokenizer=tokenizer,
    output_dir="./sft_model/balanced_heur_aug"
)
balanced_trainer.train()

Applying formatting function to train dataset:   0%|          | 0/1876 [00:00<?, ? examples/s]

Converting train dataset to ChatML:   0%|          | 0/1876 [00:00<?, ? examples/s]

Applying chat template to train dataset:   0%|          | 0/1876 [00:00<?, ? examples/s]

Tokenizing train dataset:   0%|          | 0/1876 [00:00<?, ? examples/s]

Truncating train dataset:   0%|          | 0/1876 [00:00<?, ? examples/s]

Step,Training Loss
10,1.226200
20,1.415800
30,1.214500
40,1.652200
50,1.345500
60,1.311500
70,1.697000
80,1.195300
90,1.338000
100,1.380700


TrainOutput(global_step=351, training_loss=0.8384678409649775, metrics={'train_runtime': 997.3634, 'train_samples_per_second': 5.643, 'train_steps_per_second': 0.352, 'total_flos': 2352865562910720.0, 'train_loss': 0.8384678409649775})

In [119]:
output_dir = '/content/drive/MyDrive/InkubaLM/outputs/'

In [120]:
# Save base model (optional, if not already public)
#model.save_pretrained(os.path.join(output_dir, "sft_model/balanced/base_model"))

# Save LoRA adapter
model.save_pretrained(os.path.join(output_dir, "sft_model/balanced/lora_adapter_v3"))
tokenizer.save_pretrained(os.path.join(output_dir, "sft_model/balanced/lora_adapter_v3"))


('/content/drive/MyDrive/InkubaLM/outputs/sft_model/balanced/lora_adapter_v3/tokenizer_config.json',
 '/content/drive/MyDrive/InkubaLM/outputs/sft_model/balanced/lora_adapter_v3/special_tokens_map.json',
 '/content/drive/MyDrive/InkubaLM/outputs/sft_model/balanced/lora_adapter_v3/tokenizer.model',
 '/content/drive/MyDrive/InkubaLM/outputs/sft_model/balanced/lora_adapter_v3/added_tokens.json',
 '/content/drive/MyDrive/InkubaLM/outputs/sft_model/balanced/lora_adapter_v3/tokenizer.json')

In [128]:
from datasets import concatenate_datasets

# Combine the tuple into a list
test_dataset_list = list(test_dataset)

# Concatenate all the datasets into one
combined_test_dataset = concatenate_datasets(test_dataset_list)

In [129]:
combined_test_dataset

Dataset({
    features: ['ID', 'task', 'langs', 'data_source', 'instruction', 'inputs', 'targets', 'premise'],
    num_rows: 900
})

In [130]:
v2_path = os.path.join(output_dir, "sft_model/balanced_supp/checkpoint-351")
os.makedirs(v2_path, exist_ok=True)
#inference_model = AutoModelForCausalLM.from_pretrained(ckpt_path,device_map="auto")
inference_model = model.eval()
results_df = multitask_v2.apply_inference_to_test_data(inference_model, tokenizer, combined_test_dataset)


Generating Responses:   0%|          | 0/900 [00:00<?, ?it/s]

In [131]:
results_df[['ID', 'Response']].to_csv(os.path.join(v2_path,'submission_full_finetune_v3.csv'), index=False)
results_df[['ID', 'generated','Response']].head()

,ID,generated,Response
0,ID_f3c74c7b_sentiment_test__hausa,Korau Korau Korau Korau Korau Korau Kora,2
1,ID_aad19dbf_sentiment_test__hausa,Kyakkyawa Kyakkyawa Kyakkyawa Kyakkyawa Kyakkyawa,0
2,ID_f6de0381_sentiment_test__hausa,Tsaka-tsaki Tsaka-tsaki Tsaka-tsaki Tsaka-tsaki,1
3,ID_cbec84fe_sentiment_test__swahili,Chanya Chanya Chanya Chanya Chanya Chanya Chan...,0
4,ID_885caf5c_sentiment_test__hausa,Korau Korau Korau Korau Korau Korau Kora,2


In [132]:
submission_csv = results_df[['ID','Response']]

In [43]:
submission_csv.to_csv(os.path.join(v2_path,'submission_full_finetune_v3.csv'), index=False)

In [133]:
test_csv = pd.read_csv(os.path.join(v2_path,'submission_full_finetune_v3.csv'))

In [134]:
test_csv

,ID,Response
0,ID_f3c74c7b_sentiment_test__hausa,2
1,ID_aad19dbf_sentiment_test__hausa,0
2,ID_f6de0381_sentiment_test__hausa,1
3,ID_cbec84fe_sentiment_test__swahili,0
4,ID_885caf5c_sentiment_test__hausa,2
...,...,...
895,ID_88cd08fe_test__afrixnli_hau,0
896,ID_404bc9af_test__afrixnli_swa,0
897,ID_a399ed32_test__afrixnli_swa,0
898,ID_9f28a032_test__afrixnli_hau,0
